In [442]:
from pandas import DataFrame
import pandas as pd
import random
import datetime

In [443]:
due_product_list = list()
dye_names = [ "8E","P","M","D","H"]
work_days = 5
week_start_date = datetime.datetime(2021, 8, 30)
capacity_per_day = {
"8E":288000,
"P":57600,
"M":100800,
"D":72000,
"H":86400,
}
week_days = list(
    week_start_date + datetime.timedelta(days=i) for i in range(work_days)
)
week_days

[datetime.datetime(2021, 8, 30, 0, 0),
 datetime.datetime(2021, 8, 31, 0, 0),
 datetime.datetime(2021, 9, 1, 0, 0),
 datetime.datetime(2021, 9, 2, 0, 0),
 datetime.datetime(2021, 9, 3, 0, 0)]

In [444]:
def random_requirement_generator(size: int) -> dict:
    for i in range(size):
        yield {
            "part_id": i+1,
            "dye":random.choice(dye_names),
            "cust": random.choice(["Autoliv","KSS","TRW"]),
            "safety_stock":0,
            "cards":random.randint(0,5),
            "cards_quantity":random.randint(0,14401),
        }
        
requirments = list(req for req in random_requirement_generator(10000))

In [445]:
requirements_df = pd.DataFrame.from_records(data=requirments)

requirements_df["build_quantity"] = list(
    row["cards"]*row["cards_quantity"] for index, row in requirements_df.iterrows() 
)
requirements_df["dye_capacity"] = list (
    capacity_per_day[row["dye"]]*work_days for index, row in requirements_df.iterrows()
)

total_build_quantity = requirements_df.groupby(by=requirements_df["dye"]) \
                        .sum() \
                        .reset_index() \
                        .drop(
                            columns=["part_id","safety_stock","cards","cards_quantity","dye_capacity"]
                        ) \
                        .rename({
                            "build_quantity":"total_build_qty"
                        })
requirements_df

,part_id,dye,cust,safety_stock,cards,cards_quantity,build_quantity,dye_capacity
0,1,D,Autoliv,0,1,13206,13206,360000
1,2,8E,TRW,0,1,12579,12579,1440000
2,3,8E,Autoliv,0,2,9575,19150,1440000
3,4,H,TRW,0,4,1451,5804,432000
4,5,H,Autoliv,0,1,13015,13015,432000
...,...,...,...,...,...,...,...,...
9995,9996,D,Autoliv,0,1,10522,10522,360000
9996,9997,H,Autoliv,0,1,14161,14161,432000
9997,9998,M,TRW,0,4,8416,33664,504000
9998,9999,8E,KSS,0,3,5126,15378,1440000


In [446]:

total_build_quantity['capacity_per_day'] = list(capacity_per_day[row["dye"]] for index, row in total_build_quantity.iterrows())
total_build_quantity['capacity_per_week'] = list(capacity_per_day[row["dye"]]*work_days for index, row in total_build_quantity.iterrows())
total_build_quantity

,dye,build_quantity,capacity_per_day,capacity_per_week
0,8E,36430431,288000,1440000
1,D,36193391,72000,360000
2,H,34133840,86400,432000
3,M,33533995,100800,504000
4,P,37224705,57600,288000


In [447]:
end = requirements_df.loc[requirements_df['dye'] == '8E']
princess = requirements_df.loc[requirements_df['dye'] == 'P']
maggie = requirements_df.loc[requirements_df['dye'] == 'M']
dyeHard = requirements_df.loc[requirements_df['dye'] == 'D']
high5 = requirements_df.loc[requirements_df['dye'] == 'H']
end

,part_id,dye,cust,safety_stock,cards,cards_quantity,build_quantity,dye_capacity
1,2,8E,TRW,0,1,12579,12579,1440000
2,3,8E,Autoliv,0,2,9575,19150,1440000
7,8,8E,Autoliv,0,4,4448,17792,1440000
22,23,8E,Autoliv,0,4,1603,6412,1440000
26,27,8E,Autoliv,0,1,1271,1271,1440000
...,...,...,...,...,...,...,...,...
9987,9988,8E,TRW,0,4,5891,23564,1440000
9988,9989,8E,KSS,0,4,13928,55712,1440000
9990,9991,8E,TRW,0,0,5897,0,1440000
9991,9992,8E,TRW,0,2,12583,25166,1440000


In [448]:


def scheduler(df: DataFrame) -> list[dict]:
    return pd.DataFrame.from_records(data=list(record for record in date_record_generator(df)))

def dict_merger(index:int, i:int, rows:int, qty:int, skip_days:int=0) -> dict:
    return {
            "index":index,
            "Date": week_days[i] + datetime.timedelta(days=skip_days),
            "part_id" : rows["part_id"],
            "dye": rows["dye"],
            "cards": rows["cards"],
            "cards_quantity": rows["cards_quantity"],
            "build_quantity": qty
    }

def due_product_dict(index:int, i:int, rows:int, qty:int):
    return {
            "index":index,
            "part_id" : rows["part_id"],
            "dye": rows["dye"],
            "cards": rows["cards"],
            "cards_quantity": rows["cards_quantity"],
            "build_quantity": qty
    }

def date_record_generator(df: DataFrame) -> dict:
    i=0
    sum=0
    for index, rows in df.iterrows():
        if i <=len(week_days):
            sum += rows["build_quantity"]
            if sum > capacity_per_day[rows["dye"]]:
                difference = sum-capacity_per_day[rows["dye"]]
                split = rows["build_quantity"] - difference
                yield dict_merger(index, i, rows, split)
                i += 1
                sum = difference
                if i >= len(week_days):
                    # yield dict_merger(index, i-1, rows, difference, skip_days=7-work_days)
                    due_product_list.append(due_product_dict(index, i, rows, difference))
                    break
                else:
                    yield dict_merger(index, i, rows, difference)
            else:
                yield dict_merger(index, i, rows, rows["build_quantity"])
        else:
            due_product_list.append(due_product_dict(index, i, rows, rows["build_quantity"]))
            
            

In [449]:
e_scheduled = pd.DataFrame.from_records(data=scheduler(end)).to_csv('./8End.csv')
p_scheduled = pd.DataFrame.from_records(data=scheduler(princess)).to_csv('./Princess.csv')
m_scheduled = pd.DataFrame.from_records(data=scheduler(maggie)).to_csv('./Maggie.csv')
d_scheduled = pd.DataFrame.from_records(data=scheduler(dyeHard)).to_csv('./DyeHard.csv')
h_scheduled = pd.DataFrame.from_records(data=scheduler(high5)).to_csv('./High5.csv')
due_product_dataframe = pd.DataFrame.from_records(data=due_product_list).to_csv('./Due_products.csv')
